# GraphGuard

***Locate and find Classes in Apks with updated Obfuscation Mapping***


Processing Steps:
1. String Matcher (Finding Classes and Methods
  * Counting Strings used in Classes and Methods and try to find exact matching counter.
  * Find Classes by identifying Strings used only in this single Class.
2. Structure Matcher (Finding Classes)
  * Modifiers of class
  * Modifiers, Parameters, Parameter Types and Return Types of Methods
  * Number and Types of Fields.
3. Method Matcher (Find Methods from matching Classes)
  * Modifiers
  * Return Type, Parameter Types
  * Bytecode Length
  * References to and from


***If you use Akrolyb, please have a look at [Akrolyb Interoptability](#Interoptability-with-Akrolyb)***

In [1]:
%matplotlib notebook

%load_ext autoreload
%autoreload

from IPython.core.display import display, HTML
display(HTML("<style>div.output_area pre {white-space: pre;}</style>"))

In [2]:
from core.start import process_files
from utils.formats import *

from core.accumulator import Accumulator
from core.decs import *

from strategies import\
    strings as strings_strategy,\
    methods as methods_strategy,\
    structures as structures_strategy

from utils import generate

# Loading Androguard

The following code loads the files and starts Androguard

It should support multiprocessing, however the Pipe communication seems to break when transmitting the processed Androguard Objects. I suspect the Object is simply too big for Pickle to serialize or another component in the transmitting chain.

In [3]:
AG_SESSION_FILE = "./Androguard.ag"
MULTIPROCESS_FILES = False  # Currently not working due to serialization issues


# Strategy Rules & Parameters
strings_strategy.MAX_USAGE_COUNT_STR = 20
strings_strategy.UNIQUE_STRINGS_MAJORITY = 2 / 3

methods_strategy.MIN_MATCH_POINTS = 2



# APK Files to load
file_paths = (
    "../../../Downloads/com.snapchat.android_10.85.5.74-2067_minAPI19(arm64-v8a)(nodpi)_apkmirror.com.apk",
    "../../../Downloads/com.snapchat.android_10.86.5.61-2069_minAPI19(arm64-v8a)(nodpi)_apkmirror.com.apk"
)

In [4]:
(a, d, dx), (a2, d2, dx2) = process_files(file_paths, MULTIPROCESS_FILES)

Loading Session from Apk at ../../../Downloads/com.snapchat.android_10.85.5.74-2067_minAPI19(arm64-v8a)(nodpi)_apkmirror.com.apk
Loading Session from Apk at ../../../Downloads/com.snapchat.android_10.86.5.61-2069_minAPI19(arm64-v8a)(nodpi)_apkmirror.com.apk


# List of Classes, Methods and Variables

Defining Elements that GraphGuard should try to match (obviously requires full class names).
* `c_decs` in form of a tuple of Strings
* `m_decs` in form of a tuple of MethodDeclarations
* `f_decs` in form of a tuple of FieldDeclarations

In [5]:
# Classes of the Methods and Fields to find are automatically merged with c_decs
c_decs = tuple()

In [6]:
m_decs = (
    MethodDec("rD5", "a", "rD5", "qD5"),
    MethodDec("MSg", "j0", "SGd"),
    MethodDec("xke", "g", "PV4"),
    MethodDec("GIb", "<init>", skip_params=True)
)

In [7]:
f_decs = tuple()

In [8]:
orig_c_decs = c_decs

# All c_decs that need to be resolved
c_decs = tuple(map(lambda m: m.class_name, m_decs)) + tuple(map(lambda v: v.class_name, f_decs)) + c_decs

# Processing and Matching

Loading the accumulator, an object that manages all the possible candidates that are matched by the different Matchers, and extracts the matching candidates. It also performs Inner joins on previous candidates to find the exact (or optimal) match.

In [9]:
accumulator = Accumulator()

Resolving the previously defined MethodDecs. If this fails, the MethodDecs are wrong and contain an error. Make sure the method specified with the MethodDec exists.

In [10]:
dec_cas = resolve_classes(dx, c_decs)

r_cas = tuple(dec_cas.values())
r_mas = resolve_methods(m_decs, dec_cas)
r_fas = resolve_fields(f_decs, dec_cas)


print("Resolved all Classes, Methods and Fields")
if False:
    print("", *map(pretty_format_ma, r_mas), sep="\n* ")

# Arguments to provide to Strategies
s_args = (dx, dx2, r_cas, r_mas, r_fas, accumulator)

Resolved all Classes, Methods and Fields


## String Strategy

### Exact Counter Match

Extracts Strings used either in the given methods directly or in the classes the methods define for both, the old version and the new version. It then compares the Counters for classes and methods and tries to find exact matches between the Counter Objects.

In [11]:
string_s = strings_strategy.StringStrategy(*s_args)
candidates_cs, candidates_ms = string_s.compare_counters()

accumulator.add_candidates(candidates_cs, candidates_ms)

+ Found single candidate for Method. Considering it a match 
	rD5#a(rD5, qD5) -> QE5#a(QE5, PE5)
+ Found single candidate for Method. Considering it a match 
	MSg#j0(SGd) -> V0h#j0(IMd)
+ Matching Class of single candidate method match 
	LrD5; -> LQE5;
+ Matching Class of single candidate method match 
	LMSg; -> LV0h;
Could resolve 2 new Classes, 2 new Methods


### Unique Strings

Gather all Strings that are used only in a single class ("Unique Strings") that we still need to match. Then try to find the matching class by only searching for the Unique Strings.

In [12]:
candidates_cs = string_s.compare_unique_strings(accumulator.get_unmatched_cs(r_cas))

accumulator.add_candidates(candidates_cs)

+ Found single candidate for Class. Considering it a match! 
	Lxke; -> Lvre;
Could resolve 1 new Classes, 0 new Methods


## Structure Matcher

Iterating through every single class and checks for each unmatched class if both have a similar "Profile":
* Number of Methods and Fields
* Types of Fields and Descriptors of Methods

In [13]:
structure_s = structures_strategy.StructureStrategy(*s_args)
candidates_cs = structure_s.get_exact_structure_matches()

accumulator.add_candidates(candidates_cs)

+ Found single candidate for Class. Considering it a match! 
	LGIb; -> LFOb;
Could resolve 1 new Classes, 0 new Methods


## Method Matcher

Uses different weighted criteria to get exact or optimal matches. The criteria are:
* Modifiers
* Return Type and Parameter Types
* Length of Bytecode
* References to and from

In [14]:
method_s = methods_strategy.MethodStrategy(*s_args)

print("Exact Matching")
print()

# Exact Matches
candidates_ms = method_s.try_resolve_ms(accumulator.matching_cs, True)
accumulator.add_candidates(candidates_ms=candidates_ms)

print()
print("Non-Exact Matching")
print()

# Non-Exact Matches by using weights on the criteria
candidates_ms = method_s.try_resolve_ms(accumulator.matching_cs, False)
accumulator.add_candidates(candidates_ms=candidates_ms)

Exact Matching

+ Found single candidate for Method. Considering it a match 
	GIb#<init>(java.lang.String, boolean, java.lang.String, java.lang.String, java.lang.String, java.lang.Long, Ds5, long, boolean, java.lang.Long, long) -> FOb#<init>(java.lang.String, boolean, java.lang.String, java.lang.String, java.lang.String, java.lang.Long, Xt5, long, boolean, java.lang.Long, long)
Could resolve 0 new Classes, 1 new Methods

Non-Exact Matching

+ Found single candidate for Method. Considering it a match 
	xke#g(PV4) -> vre#g(pX4)
+ Found single candidate for Method. Considering it a match 
	GIb#<init>(java.lang.String, boolean, java.lang.String, java.lang.String, java.lang.String, java.lang.Long, Ds5, long, boolean, java.lang.Long, long) -> FOb#<init>(java.lang.String, boolean, java.lang.String, java.lang.String, java.lang.String, java.lang.Long, Xt5, long, boolean, java.lang.Long, long)
Could resolve 0 new Classes, 1 new Methods


In [15]:
for fa in r_fas:
    ef = fa.get_field()
    f_class = ef.get_class_name()
    if f_class in accumulator.matching_cs:
        f2_class = accumulator.matching_cs[f_class]
    
        ca2 = dx2.get_class_analysis(f2_class)
        from androguard.core.bytecodes.dvm import EncodedField
        from androguard.core.analysis.analysis import FieldAnalysis
        cfs = (
            (lambda f: f.get_field().get_access_flags_string(), 10),
            (lambda f: f.get_field().get_descriptor(), 10),
            (lambda f: f.get_field().get_size(), 5),
            (lambda f: len(f.get_xref_read()), 4),
            (lambda f: len(f.get_xref_write()), 4)
        )
        
        scores = {fa2: sum(((cf(fa) == cf(fa2)) * score) for cf, score in cfs) for fa2 in ca2.get_fields()}
        m = max(scores.values())
        scores = filter(lambda x: x[1] == m, scores.items())
        
        print("Best Matches for Field", pretty_format_fa(ef), "in Class", f2_class, "-", ", ".join(tuple(map(lambda x: str(x[0].name), scores))))

# Results
Display summary and matching pairs.

Output to compatible MethodDec input to update from auto-updated values.

In [16]:
print(len(accumulator.matching_cs), "/", len(r_cas), "Classes were matched")
print(len(accumulator.matching_ms), "/", len(m_decs), "Methods were matched")

print()
print("Matching Classes:")
for c1, c2 in accumulator.matching_cs.items():
    print("•", pretty_format_class(c1), "->", pretty_format_class(c2))

print()
print("Matching Methods: ")
for m, ma in accumulator.matching_ms.items():
    print("•", pretty_format_ma(m), "->", pretty_format_ma(ma))

4 / 4 Classes were matched
4 / 4 Methods were matched

Matching Classes:
• rD5 -> QE5
• MSg -> V0h
• xke -> vre
• GIb -> FOb

Matching Methods: 
• rD5#a(rD5, qD5) -> QE5#a(QE5, PE5)
• MSg#j0(SGd) -> V0h#j0(IMd)
• GIb#<init>(java.lang.String, boolean, java.lang.String, java.lang.String, java.lang.String, java.lang.Long, Ds5, long, boolean, java.lang.Long, long) -> FOb#<init>(java.lang.String, boolean, java.lang.String, java.lang.String, java.lang.String, java.lang.Long, Xt5, long, boolean, java.lang.Long, long)
• xke#g(PV4) -> vre#g(pX4)


In [17]:
generate.generate_decs(r_mas, accumulator.matching_ms)

m_decs = (
	MethodDec('QE5', 'a', 'QE5', 'PE5'),
	MethodDec('V0h', 'j0', 'IMd'),
	MethodDec('vre', 'g', 'pX4'),
	MethodDec('FOb', '<init>', 'java.lang.String', 'boolean', 'java.lang.String', 'java.lang.String', 'java.lang.String', 'java.lang.Long', 'Xt5', 'long', 'boolean', 'java.lang.Long', 'long')
)


# Interoptability with Akrolyb

The following Code snippet allows to "extract" MethodDecs from MemberDeclarations in Akrolyb. This automates the process of providing GraphGuard the hooks it should find. Just paste the output (tuple of MethodDecs) in the corresponding section in this Notebook.

## Extract

It is not in Python, since it would require a Kotlin Parser and evaluating imports, variables etc. Just executing the Constructors in Kotlin, then getting the values is much easier than static analysis. 

The `main` function can (and should) be run statically (locally on the machine and not on your Android) to build the list of `MethodDec`s that GraphGuard is supposed to match in an updated build of the target application. Adjust the MemberDeclarations Class to the Class where you declared the `MemberDeclaration`s.

Note: This code does obviously not know if you left out the parameters on purpose (a lot of parameter types, or multiple constructors...). Please modify the results for the Methods concerned by using the optional `skip_params=True` argument for `MethodDec`.

```kotlin
fun main() {
    fun MemberDec.formatToGraphGuard() = buildString {
        append("MethodDec('")
        append(classDec.className)
        append("', '")
        append(if (this@formatToGraphGuard is MethodDec) methodName else "<init>")
        append("'")
        for (param in params) {
            append(", '")
            if (param is String)
                append(param)
            else if (param is ClassDec)
                append(param.className)
            else if (param is Class<*>)
                append(param.name)
            else error("Illegal Type for param")
            append("'")
        }
        append(")")
    }
    
    println("decs_to_find = (")
    val strs = MemberDeclarations::class.java.declaredFields.mapNotNull {
        it.isAccessible = true
        val x = it.get(MemberDeclarations)
        if (x !is MemberDec) return@mapNotNull null

        x.formatToGraphGuard()
    }.joinToString(separator = ",\n\t", prefix = "\t")
    println(strs)
    println(")")
}
```